# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 4:30PM,261455,16,9396709,Sartorius Lab Products and Service,Released
1,Apr 19 2023 4:20PM,261453,16,9396676,Sartorius Lab Products and Service,Released
2,Apr 19 2023 4:19PM,261452,19,RAJU41923,"Dr. Raju's Pharma Solutions, LLC",Released
3,Apr 19 2023 4:15PM,261450,16,9396674,Sartorius Lab Products and Service,Released
4,Apr 19 2023 4:08PM,261445,16,9396663,Sartorius Lab Products and Service,Released
5,Apr 19 2023 4:08PM,261445,16,9396672,Sartorius Lab Products and Service,Released
6,Apr 19 2023 4:05PM,261444,10,HH9264,Hush Hush,Executing
7,Apr 19 2023 4:04PM,261442,22,4212023,FUJIFILM Diosynth Biotechnologies Texas LLC,Executing
8,Apr 19 2023 4:01PM,261441,19,SNAP36226002,"Snap Medical Industries, LLC",Released
9,Apr 19 2023 2:56PM,261433,20,ATR-I000117,"HVL, LLC dba Atrium Innovations",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,261445,Released,2
20,261450,Released,1
21,261452,Released,1
22,261453,Released,1
23,261455,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261445,NaN,NaN,2.0
261450,NaN,NaN,1.0
261452,NaN,NaN,1.0
261453,NaN,NaN,1.0
261455,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260910,8.0,1.0,4.0
261117,6.0,3.0,1.0
261398,28.0,11.0,3.0
261405,0.0,0.0,2.0
261406,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260910,8,1,4
261117,6,3,1
261398,28,11,3
261405,0,0,2
261406,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260910,8,1,4
1,261117,6,3,1
2,261398,28,11,3
3,261405,0,0,2
4,261406,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260910,8,1,4
1,261117,6,3,1
2,261398,28,11,3
3,261405,,,2
4,261406,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 4:30PM,261455,16,Sartorius Lab Products and Service
1,Apr 19 2023 4:20PM,261453,16,Sartorius Lab Products and Service
2,Apr 19 2023 4:19PM,261452,19,"Dr. Raju's Pharma Solutions, LLC"
3,Apr 19 2023 4:15PM,261450,16,Sartorius Lab Products and Service
4,Apr 19 2023 4:08PM,261445,16,Sartorius Lab Products and Service
6,Apr 19 2023 4:05PM,261444,10,Hush Hush
7,Apr 19 2023 4:04PM,261442,22,FUJIFILM Diosynth Biotechnologies Texas LLC
8,Apr 19 2023 4:01PM,261441,19,"Snap Medical Industries, LLC"
9,Apr 19 2023 2:56PM,261433,20,"HVL, LLC dba Atrium Innovations"
10,Apr 19 2023 2:39PM,261432,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 19 2023 4:30PM,261455,16,Sartorius Lab Products and Service,,,1
1,Apr 19 2023 4:20PM,261453,16,Sartorius Lab Products and Service,,,1
2,Apr 19 2023 4:19PM,261452,19,"Dr. Raju's Pharma Solutions, LLC",,,1
3,Apr 19 2023 4:15PM,261450,16,Sartorius Lab Products and Service,,,1
4,Apr 19 2023 4:08PM,261445,16,Sartorius Lab Products and Service,,,2
5,Apr 19 2023 4:05PM,261444,10,Hush Hush,,1,
6,Apr 19 2023 4:04PM,261442,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
7,Apr 19 2023 4:01PM,261441,19,"Snap Medical Industries, LLC",,,1
8,Apr 19 2023 2:56PM,261433,20,"HVL, LLC dba Atrium Innovations",,,1
9,Apr 19 2023 2:39PM,261432,19,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 4:30PM,261455,16,Sartorius Lab Products and Service,1,,
1,Apr 19 2023 4:20PM,261453,16,Sartorius Lab Products and Service,1,,
2,Apr 19 2023 4:19PM,261452,19,"Dr. Raju's Pharma Solutions, LLC",1,,
3,Apr 19 2023 4:15PM,261450,16,Sartorius Lab Products and Service,1,,
4,Apr 19 2023 4:08PM,261445,16,Sartorius Lab Products and Service,2,,
5,Apr 19 2023 4:05PM,261444,10,Hush Hush,,1,
6,Apr 19 2023 4:04PM,261442,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
7,Apr 19 2023 4:01PM,261441,19,"Snap Medical Industries, LLC",1,,
8,Apr 19 2023 2:56PM,261433,20,"HVL, LLC dba Atrium Innovations",1,,
9,Apr 19 2023 2:39PM,261432,19,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 4:30PM,261455,16,Sartorius Lab Products and Service,1,,
1,Apr 19 2023 4:20PM,261453,16,Sartorius Lab Products and Service,1,,
2,Apr 19 2023 4:19PM,261452,19,"Dr. Raju's Pharma Solutions, LLC",1,,
3,Apr 19 2023 4:15PM,261450,16,Sartorius Lab Products and Service,1,,
4,Apr 19 2023 4:08PM,261445,16,Sartorius Lab Products and Service,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 4:30PM,261455,16,Sartorius Lab Products and Service,1.0,NaN,NaN
1,Apr 19 2023 4:20PM,261453,16,Sartorius Lab Products and Service,1.0,NaN,NaN
2,Apr 19 2023 4:19PM,261452,19,"Dr. Raju's Pharma Solutions, LLC",1.0,NaN,NaN
3,Apr 19 2023 4:15PM,261450,16,Sartorius Lab Products and Service,1.0,NaN,NaN
4,Apr 19 2023 4:08PM,261445,16,Sartorius Lab Products and Service,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 4:30PM,261455,16,Sartorius Lab Products and Service,1.0,0.0,0.0
1,Apr 19 2023 4:20PM,261453,16,Sartorius Lab Products and Service,1.0,0.0,0.0
2,Apr 19 2023 4:19PM,261452,19,"Dr. Raju's Pharma Solutions, LLC",1.0,0.0,0.0
3,Apr 19 2023 4:15PM,261450,16,Sartorius Lab Products and Service,1.0,0.0,0.0
4,Apr 19 2023 4:08PM,261445,16,Sartorius Lab Products and Service,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,261444,0.0,1.0,0.0
16,1045803,5.0,0.0,0.0
19,1307128,8.0,11.0,28.0
20,1306276,11.0,7.0,16.0
22,261442,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,261444,0.0,1.0,0.0
1,16,1045803,5.0,0.0,0.0
2,19,1307128,8.0,11.0,28.0
3,20,1306276,11.0,7.0,16.0
4,22,261442,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,0.0,1.0,0.0
1,16,5.0,0.0,0.0
2,19,8.0,11.0,28.0
3,20,11.0,7.0,16.0
4,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,0.0
1,16,Released,5.0
2,19,Released,8.0
3,20,Released,11.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,22
Status,,,,,
Completed,0.0,0.0,28.0,16.0,0.0
Executing,1.0,0.0,11.0,7.0,1.0
Released,0.0,5.0,8.0,11.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,22
0,Completed,0.0,0.0,28.0,16.0,0.0
1,Executing,1.0,0.0,11.0,7.0,1.0
2,Released,0.0,5.0,8.0,11.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,22
0,Completed,0.0,0.0,28.0,16.0,0.0
1,Executing,1.0,0.0,11.0,7.0,1.0
2,Released,0.0,5.0,8.0,11.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()